# CTM

## Preprocessing

Applico un preprocessing diverso da LDA e ProdLDA in quanto CTM ne richiede molto  meno.

In [1]:
file_path = r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\POI\UniPerugia_POI.csv'
with open(file_path, 'r') as file:
    poi = file.readlines()

print(poi)

['id;name;description;id_openstreetmap;nome;descrizione;latitude;longitude\n', "1;Villalba Park;the villalba park is a wooded area of over 20 ha the park is situated at an altitude between 600 and 700 meters and it was designed by the mountain communities as park facilities the area situated at the edge of the monte rufeno nature reserve inside the vast state owned forest of selva di meana is remarkable from an environmental point of view the park is included in the ample interregional park the woods of tall trees include species of oak with some other rarer but important varieties such as maple ash beech and chestnut trees among the shrubs there are the rare fraxinella ;2959161;Parco di Villalba;Area boschiva di oltre 20 ha. situata ad un'altitudine fra i 600 ed i 700 metri e destinata dalla ComunitA  Montana a parco attrezzato.;4282034852;1190938726\n", '2;Prati di Stroncone;a dreamlike destination about 1000 metres above sea level near stroncone a medieval village perched on a hill 

In [2]:
import pandas as pd

# Carica il file CSV
df = pd.read_csv(file_path, sep=';')

# Seleziona solo le colonne 'nome' e 'descrizione' in italiano
df_italiano = df[['id', 'latitude', 'longitude', 'nome', 'descrizione']]

file_save = r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\POI\UniPerugia_POI_italiano.csv'
# Salva il dataframe con le sole colonne italiane in un nuovo file CSV
df_italiano.to_csv(file_save, index=False)

In [3]:
descriptions = df_italiano['descrizione'].replace('\n', '')
print(descriptions)

0      Area boschiva di oltre 20 ha. situata ad un'al...
1      Vicino a Stroncone, borgo medievale arroccato ...
2      Le Gole del Nera, dove la natura, la cultura e...
3      La rocca si trova nell'isola Polvese che, insi...
4      LaIsola Maggiore A" la seconda per grandezza d...
                             ...                        
570    Unaopera d'arte "ziggurart"di uno dei numerosi...
571    Uno spazio espositivo, punto di arrivo della c...
572                                 Descrizione mancante
573    Un antico castello immerso in un bosco millenario
574                                 Descrizione mancante
Name: descrizione, Length: 575, dtype: object


In [4]:
file_corpus = r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\my_corpus_folder\my_corpus.txt'
with open(file_corpus, 'w') as f:
    for description in descriptions:
        if isinstance(description, str):
            description = description.replace('\n', ' ')
            f.write(description + '\n')

In [5]:
# carica il corpus
# Apre il file di testo in modalità di lettura ('r')
with open(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\my_corpus_folder\my_corpus.txt', 'r') as file:
    # Legge il contenuto del file e lo memorizza in una variabile
    corpus = file.read()

# Stampa il contenuto del file
print(corpus)

Area boschiva di oltre 20 ha. situata ad un'altitudine fra i 600 ed i 700 metri e destinata dalla ComunitA  Montana a parco attrezzato.
Vicino a Stroncone, borgo medievale arroccato su un colle ricoperto di uliveti e gelosamente protetto dalle solide mura castellane, si cela un vero e proprio luogo da sogno, a circa mille metri di altitudine.
Le Gole del Nera, dove la natura, la cultura e la storia si incontrano da oltre duemila anni in un'atmosfera unica, sono una degna prosecuzione della Valnerina dopo l'attraversamento della Conca Ternana del fiume Nera.
La rocca si trova nell'isola Polvese che, insieme all'isola Maggiore e alla Minore, occupa il bacino del Lago Trasimeno.
LaIsola Maggiore A" la seconda per grandezza del lago Trasimeno e si presenta ricoperta da una fitta vegetazione di olivi, lecci e cipressi. A differenza dellaIsola Polvese e della Minore A" ancora oggi urbanizzata; presenta infatti un piccolo villaggio sulla riva occidentale.
Per raggiungere il santuario della Ma

In [6]:
corpus = list(corpus.split())

In [7]:
from octis.dataset.dataset import Dataset

class MyDataset(Dataset):
    def __init__(self, data_folder):
        super(MyDataset, self).__init__(data_folder)

    def _load_dataset(self):
        with open(self.data_folder, 'r') as f:
            # leggi il file di corpus e crea una lista di documenti
            documents = f.read().splitlines()
        # restituisci la lista di documenti come dataset
        return documents

In [8]:
my_corpus_folder = r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\my_corpus_folder'
dataset = MyDataset(my_corpus_folder)

In [9]:
import os
import string
from octis.preprocessing.preprocessing import Preprocessing

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
dataset_path = r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\dataset\dataset_CTM'
dataset_save = dataset_path

In [29]:
import os
import string
from octis.preprocessing.preprocessing import Preprocessing
os.chdir(os.path.pardir)

# Initialize preprocessing
preprocessor = Preprocessing(vocabulary=None, max_features=None,
                             remove_punctuation=True,
                             lemmatize=False, split=False, language="italian",
                             min_chars=3, min_words_docs=0)
# preprocess
dataset = preprocessor.preprocess_dataset(file_corpus)

# save the preprocessed dataset
dataset.save(dataset_save)

100%|██████████| 573/573 [00:00<00:00, 44065.57it/s]

created vocab
3197


In [30]:
print(len(dataset.get_vocabulary()))
print(len(dataset.get_corpus()))   # numero di POI

3197
573


## Optimization

In [31]:
from octis.models.CTM import CTM
from octis.dataset.dataset import Dataset
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity

In [32]:
dataset_save = r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\dataset\dataset_CTM'

In [33]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(dataset_save)

In [34]:
dataset.get_corpus()

[['area',
  'boschiva',
  'oltre',
  'situata',
  'altitudine',
  'fra',
  'metri',
  'destinata',
  'dalla',
  'comunita',
  'montana',
  'parco',
  'attrezzato'],
 ['vicino',
  'stroncone',
  'borgo',
  'medievale',
  'arroccato',
  'colle',
  'ricoperto',
  'uliveti',
  'gelosamente',
  'protetto',
  'dalle',
  'solide',
  'mura',
  'castellane',
  'cela',
  'vero',
  'proprio',
  'luogo',
  'sogno',
  'circa',
  'mille',
  'metri',
  'altitudine'],
 ['gole',
  'del',
  'nera',
  'dove',
  'natura',
  'cultura',
  'storia',
  'incontrano',
  'oltre',
  'duemila',
  'anni',
  'atmosfera',
  'unica',
  'sono',
  'una',
  'degna',
  'prosecuzione',
  'della',
  'valnerina',
  'dopo',
  'attraversamento',
  'della',
  'conca',
  'ternana',
  'del',
  'fiume',
  'nera'],
 ['rocca',
  'trova',
  'nell',
  'isola',
  'polvese',
  'che',
  'insieme',
  'all',
  'isola',
  'maggiore',
  'alla',
  'minore',
  'occupa',
  'bacino',
  'del',
  'lago',
  'trasimeno'],
 ['laisola',
  'maggiore',


In [86]:
from sentence_transformers import SentenceTransformer

In [35]:
model = CTM(num_epochs=30, inference_type='zeroshot', bert_model="distiluse-base-multilingual-cased-v2",use_partitions=False)

In [36]:
model_output = model.train_model(dataset)   # train the model

In [37]:
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_npmi')
topic_diversity = TopicDiversity(topk=10)

In [39]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(model_output)
print("Topic diversity: "+str(topic_diversity_score))

npmi_score = npmi.score(model_output)
print("NPMI: "+str(npmi_score))


Topic diversity: 0.86
NPMI: -0.30178353216851994


In [40]:
model_output

{'topics': [['nell',
   'breve',
   'rinomato',
   'insoliti',
   'nicchioni',
   'spoleto',
   'lavoro',
   'muvit',
   'urne',
   'folclore'],
  ['pia',
   'sud',
   'abbazia',
   'uso',
   'tempo',
   'collina',
   'torgiano',
   'cima',
   'torrente',
   'marchigiano'],
  ['museo',
   'nato',
   'scrigno',
   'origini',
   'attivita',
   'rocca',
   'domestiche',
   'vero',
   'moderna',
   'umbria'],
  ['natura',
   'storia',
   'mondo',
   'cappelle',
   'grandi',
   'alberi',
   'raccontano',
   'macigni',
   'spina',
   'distanza'],
  ['quale',
   'che',
   'facciata',
   'costruito',
   'edificio',
   'santi',
   'piazza',
   'nucleo',
   'posizione',
   'venne'],
  ['foligno',
   'dei',
   'anni',
   'uno',
   'importanti',
   'rappresenta',
   'strada',
   'edificato',
   'park',
   'piccolo'],
  ['sinistra',
   'principale',
   'all',
   'piazza',
   'dei',
   'secolo',
   'via',
   'epoca',
   'centro',
   'chiesa'],
  ['che',
   'come',
   'sul',
   'costruzione',
   'con

In [41]:
type(model_output['topics'])

list

In [42]:
search_space = {"num_layers": Categorical({1, 2, 3}),
                "num_neurons": Categorical({100, 200, 300}),
                "activation": Categorical({'sigmoid', 'relu', 'softplus'}),
                "dropout": Real(0.0, 0.95),
                "num_topics": Integer(low=5,high=15)
}

In [43]:
optimization_runs=30
model_runs=1

In [44]:
for t in model_output['topics']:
  print(" ".join(t))

nell breve rinomato insoliti nicchioni spoleto lavoro muvit urne folclore
pia sud abbazia uso tempo collina torgiano cima torrente marchigiano
museo nato scrigno origini attivita rocca domestiche vero moderna umbria
natura storia mondo cappelle grandi alberi raccontano macigni spina distanza
quale che facciata costruito edificio santi piazza nucleo posizione venne
foligno dei anni uno importanti rappresenta strada edificato park piccolo
sinistra principale all piazza dei secolo via epoca centro chiesa
che come sul costruzione con una costruito del colle fiume
come venne chiesa trova santa del colle edificio maria mura
trova piazza sede maria palazzo nella della san convento chiesa


### Ottimizzo Topic Diversity

In [45]:
optimizer=Optimizer()
optimization_result = optimizer.optimize( 
    model, dataset, topic_diversity, search_space, number_of_call=optimization_runs,
    model_runs=model_runs, save_models=False,
    extra_metrics=None, # to keep track of other metrics
    save_path=r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_TD')
optimization_result.save_to_csv(r"C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_TD\results.csv")

Current call:  0
Current call:  1
Current call:  2
Current call:  3
Current call:  4
Current call:  5
Current call:  6
Current call:  7
Current call:  8
Current call:  9
Current call:  10
Current call:  11
Current call:  12
Current call:  13
Current call:  14
Current call:  15
Current call:  16
Current call:  17
Current call:  18
Current call:  19
Current call:  20
Current call:  21
Current call:  22
Current call:  23
Current call:  24
Current call:  25
Current call:  26
Current call:  27
Current call:  28
Current call:  29


In [47]:
import pandas as pd

df = pd.read_csv(r"C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_TD\results.csv")

# Trova l'indice del valore massimo di "mean" nella colonna "topic diversity"
indice_max_mean = df['Mean(model_runs)'].idxmax()

# Estrai i valori delle colonne corrispondenti ai parametri richiesti per l'indice trovato
parametri_max_mean = df.loc[indice_max_mean, ['Mean(model_runs)','num_iteration', 'activation', 'num_layers', 'num_neurons', 'num_topics']]

print("Parametri associati al valore massimo di mean nella colonna topic diversity:")
print(parametri_max_mean)

Parametri associati al valore massimo di mean nella colonna topic diversity:
Mean(model_runs)    0.966667
num_iteration             10
activation          softplus
num_layers                 3
num_neurons              300
num_topics                 6
Name: 10, dtype: object


In [48]:
model = CTM(num_epochs=30, inference_type='zeroshot', bert_model="distiluse-base-multilingual-cased-v2", num_topics=6, num_neurons=300, num_layers=3, activation="softplus", use_partitions=False)

In [49]:
output_TD = model.train_model(dataset)

In [50]:
for t in output_TD['topics']:
  print(" ".join(t))

chiesa nella otricoli del una situato allainterno museo che per
dove laoratorio municipale anacleto meccanismi ampia monumentali antica anni quel
come duomo dei che principale una presenta colle metri ancora
santa nei maria piazza venne pressi con del della una
atto campaniletto risale anacleto cultura snoda vallingegno diletto figlio giorgio
storia prospetto tradizionali cimabue testimonianze found pietralunga tenere antica scalo


In [51]:
topics = pd.DataFrame(output_TD['topics'])
topics.to_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_TD\topics.xlsx', index=False)  

In [52]:
vocabolario = []
with open(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\dataset\dataset_CTM\vocabulary.txt', 'r') as file:
    for word in file:
        vocabolario.append(word)

In [53]:
# Carica la matrice topic-word in un DataFrame pandas
topic_word_matrix = pd.DataFrame(output_TD['topic-word-matrix'])

# Imposta i nomi delle colonne come parole

# Ottieni i primi k termini da ciascuna riga come nomi di colonna
topic_word_matrix.columns = [vocabolario[i] for i in range(topic_word_matrix.shape[1])]

# Stampa il DataFrame
print(topic_word_matrix)

topic_word_matrix.to_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_TD\topic_word_matrix.xlsx', index=False) 

   aaaaaaacastello\n  abbadesse\n  abbandonato\n  abbastanza\n  abbazia\n   
0          -0.117280    -0.056652      -0.043873     -0.021005  -0.002550  \
1           0.035305    -0.159080      -0.083646      0.054317   0.025934   
2          -0.060751    -0.033862      -0.020362     -0.000913  -0.009362   
3          -0.042707    -0.034475       0.002767     -0.021143   0.026079   
4           0.021225     0.020103      -0.072666      0.066758  -0.128462   
5          -0.124745    -0.005264       0.050539      0.041694   0.050849   

   abbaziale\n  abbazie\n  abitati\n  abitativo\n  abitato\n  ...     xii\n   
0    -0.037414  -0.090274  -0.049364     0.002026  -0.073643  ...  0.004135  \
1    -0.015164  -0.007812   0.038281     0.034294  -0.046792  ...  0.020271   
2    -0.022659  -0.009797  -0.002775    -0.017527  -0.072927  ...  0.027979   
3    -0.155817  -0.045686  -0.013390     0.024430   0.044001  ...  0.037269   
4     0.044685   0.068697   0.042310    -0.143285  -0.048068  ...

In [54]:
import pandas as pd

corpus_processed = pd.read_csv(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\dataset\dataset_CTM\corpus.tsv', sep='\t', header=None)
corpus_processed
new_df = pd.DataFrame(corpus_processed[0])

In [55]:
num = 1
for topic in output_TD['topic-document-matrix']:
  for i in range(0,len(topic)):
    new_df.loc[i,num] = topic[i]
  num = num+1
new_df

,0,1,2,3,4,5,6
0,area boschiva oltre situata altitudine fra met...,0.078928,0.271573,0.084710,0.100128,0.201167,0.263494
1,vicino stroncone borgo medievale arroccato col...,0.327244,0.063241,0.292987,0.204307,0.053673,0.058548
2,gole del nera dove natura cultura storia incon...,0.250215,0.102578,0.217831,0.227072,0.073503,0.128801
3,rocca trova nell isola polvese che insieme all...,0.090575,0.370721,0.035465,0.114443,0.174383,0.214412
4,laisola maggiore seconda per grandezza del lag...,0.355720,0.065085,0.140371,0.335943,0.046045,0.056836
...,...,...,...,...,...,...,...
568,unaopera arte ziggurart uno dei numerosi artis...,0.028544,0.338982,0.031743,0.034491,0.424139,0.142102
569,uno spazio espositivo punto arrivo della cresc...,0.021005,0.304653,0.028066,0.052245,0.242120,0.351910
570,descrizione mancante,0.017826,0.380739,0.016899,0.039771,0.299785,0.244981
571,antico castello immerso bosco millenario,0.027539,0.340589,0.034251,0.025344,0.160732,0.411546


In [56]:
new_df.to_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_TD\topic-document-matrix.xlsx', index=False)  

### Ottimizzo Topic Coherence

In [57]:
dataset_save = r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\dataset\dataset_CTM'


In [58]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(dataset_save)

In [59]:
model = CTM(num_epochs=30, inference_type='zeroshot', bert_model="distiluse-base-multilingual-cased-v2",use_partitions=False)

In [61]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_npmi')


In [62]:
search_space = {"num_layers": Categorical({1, 2, 3}),
                "num_neurons": Categorical({100, 200, 300}),
                "activation": Categorical({'sigmoid', 'relu', 'softplus'}),
                "dropout": Real(0.0, 0.95),
                "num_topics": Integer(low=5,high=15)
}

In [63]:
optimization_runs=30
model_runs=1

In [64]:
optimizer=Optimizer()
optimization_result = optimizer.optimize(
    model, dataset, npmi, search_space, number_of_call=optimization_runs,
    model_runs=model_runs, save_models=False,
    extra_metrics=None, # to keep track of other metrics
    save_path=r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_npmi')
optimization_result.save_to_csv(r"C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_npmi\results.csv")

Current call:  0
Current call:  1
Current call:  2
Current call:  3
Current call:  4
Current call:  5
Current call:  6
Current call:  7
Current call:  8
Current call:  9
Current call:  10
Current call:  11
Current call:  12
Current call:  13
Current call:  14
Current call:  15
Current call:  16
Current call:  17
Current call:  18
Current call:  19
Current call:  20
Current call:  21
Current call:  22
Current call:  23
Current call:  24
Current call:  25
Current call:  26
Current call:  27
Current call:  28
Current call:  29


In [76]:
import pandas as pd

df = pd.read_csv(r"C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_npmi\results.csv")

# Trova l'indice del valore massimo di "mean" nella colonna "topic diversity"
indice_max_mean = df['Mean(model_runs)'].idxmax()

# Estrai i valori delle colonne corrispondenti ai parametri richiesti per l'indice trovato
parametri_max_mean = df.loc[indice_max_mean, ['Mean(model_runs)','num_iteration', 'activation', 'num_layers', 'num_neurons', 'num_topics']]

print("Parametri associati al valore massimo di mean nella colonna topic diversity:")
print(parametri_max_mean)

Parametri associati al valore massimo di mean nella colonna topic diversity:
Mean(model_runs)   -0.151218
num_iteration             21
activation              relu
num_layers                 1
num_neurons              300
num_topics                15
Name: 21, dtype: object


In [77]:
model = CTM(num_epochs=30, inference_type='zeroshot', bert_model="distiluse-base-multilingual-cased-v2", num_topics=15, num_neurons=300, num_layers=1, activation="relu", use_partitions=False)

In [78]:
output_npmi = model.train_model(dataset)

Batches: 100%|██████████| 6/6 [01:02<00:00, 10.39s/it]


In [79]:
for t in output_npmi['topics']:
  print(" ".join(t))

secolo xiv complesso arte xvi francesco convento francescano ospitata xiii
spoleto con reperti culturale norcia anni due fontana delle caratteristiche
avorio vitigno lamine preghiera etruscaa sbalzo legno rivestito placche bartolomeo
immerso tradizione ricca natura sentieri boschi mtb deruta confine passeggiate
maria santa secolo edificio chiesa assunta xii piazza venne parrocchiale
maestro palazzo anticamente dei iacovetti cardinale esattamente costruire quello church
nicola parco allainterno frazione repubblica pinacoteca visita museo nord sede
chiesa santa maria fuori facciata convento dedicata cattedrale maggiore poco
etrusca fiume tevere area necropoli sud archeologica lungo lago trasimeno
distanza scoperta antico conosciuto antiche pochi incontaminata chilometri poca mozzafiato
lago del centro palazzo anche militare urbano storico trasimeno perugia
costruita nord porta eta orvieto costruito quello est anno suo
che del come della sulla nel edificata oggi castello monastero
medieva

In [80]:
topics = pd.DataFrame(output_npmi['topics'])
topics.to_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_npmi\topics.xlsx', index=False)  

In [81]:
vocabolario = []
with open(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\dataset\dataset_CTM\vocabulary.txt', 'r') as file:
    for word in file:
        vocabolario.append(word)

In [82]:
# Carica la matrice topic-word in un DataFrame pandas
topic_word_matrix = pd.DataFrame(output_npmi['topic-word-matrix'])

# Imposta i nomi delle colonne come parole

# Ottieni i primi k termini da ciascuna riga come nomi di colonna
topic_word_matrix.columns = [vocabolario[i] for i in range(topic_word_matrix.shape[1])]

# Stampa il DataFrame
print(topic_word_matrix)

topic_word_matrix.to_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_npmi\topic_word_matrix.xlsx', index=False) 

    aaaaaaacastello\n  abbadesse\n  abbandonato\n  abbastanza\n  abbazia\n   
0            0.022940    -0.036757      -0.014231      0.032971   0.025797  \
1           -0.016182    -0.048027      -0.066371     -0.057878  -0.044079   
2            0.034812     0.060502       0.035626      0.053890   0.093745   
3            0.059521     0.062567       0.047957      0.009741  -0.082720   
4           -0.145505    -0.021406      -0.039652     -0.122532   0.009682   
5            0.061295     0.048257       0.046251     -0.003647  -0.056557   
6           -0.085054     0.028580      -0.004877     -0.020086  -0.105785   
7           -0.203303    -0.081758       0.062988     -0.131621   0.037419   
8            0.004644     0.037392      -0.047582      0.017773   0.020711   
9            0.032331     0.026081       0.034585      0.004607   0.124314   
10          -0.106828    -0.048443      -0.151894     -0.107755  -0.057486   
11          -0.020024    -0.049375      -0.125082      0.066452 

In [83]:
import pandas as pd

corpus_processed = pd.read_csv(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\dataset\dataset_CTM\corpus.tsv', sep='\t', header=None)
corpus_processed
new_df = pd.DataFrame(corpus_processed[0])

In [84]:
num = 1
for topic in output_TD['topic-document-matrix']:
  for i in range(0,len(topic)):
    new_df.loc[i,num] = topic[i]
  num = num+1
new_df

,0,1,2,3,4,5,6
0,area boschiva oltre situata altitudine fra met...,0.078928,0.271573,0.084710,0.100128,0.201167,0.263494
1,vicino stroncone borgo medievale arroccato col...,0.327244,0.063241,0.292987,0.204307,0.053673,0.058548
2,gole del nera dove natura cultura storia incon...,0.250215,0.102578,0.217831,0.227072,0.073503,0.128801
3,rocca trova nell isola polvese che insieme all...,0.090575,0.370721,0.035465,0.114443,0.174383,0.214412
4,laisola maggiore seconda per grandezza del lag...,0.355720,0.065085,0.140371,0.335943,0.046045,0.056836
...,...,...,...,...,...,...,...
568,unaopera arte ziggurart uno dei numerosi artis...,0.028544,0.338982,0.031743,0.034491,0.424139,0.142102
569,uno spazio espositivo punto arrivo della cresc...,0.021005,0.304653,0.028066,0.052245,0.242120,0.351910
570,descrizione mancante,0.017826,0.380739,0.016899,0.039771,0.299785,0.244981
571,antico castello immerso bosco millenario,0.027539,0.340589,0.034251,0.025344,0.160732,0.411546


In [85]:
new_df.to_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\risultati\CTM\result_npmi\topic-document-matrix.xlsx', index=False)  